# Projeto Automação Web - Busca de Preços

### Objetivo: treinar um projeto em que a gente tenha que usar automações web com Selenium para buscar as informações que precisamos

- Já fizemos um projeto com esse objetivo no Módulo de Python e Web e em gravações de encontros ao vivo, mas não custa nada treinar mais um pouco.

### Como vai funcionar:

- Imagina que você trabalha na área de compras de uma empresa e precisa fazer uma comparação de fornecedores para os seus insumos/produtos.

- Nessa hora, você vai constantemente buscar nos sites desses fornecedores os produtos disponíveis e o preço, afinal, cada um deles pode fazer promoção em momentos diferentes e com valores diferentes.

- Seu objetivo: Se o valor dos produtos for abaixo de um preço limite definido por você, você vai descobrir os produtos mais baratos e atualizar isso em uma planilha.
- Em seguida, vai enviar um e-mail com a lista dos produtos abaixo do seu preço máximo de compra.

- No nosso caso, vamos fazer com produtos comuns em sites como Google Shopping e Buscapé, mas a ideia é a mesma para outros sites.

### Outra opção:

- APIs

### O que temos disponível?

- Planilha de Produtos, com os nomes dos produtos, o preço máximo, o preço mínimo (para evitar produtos "errados" ou "baratos de mais para ser verdade" e os termos que vamos querer evitar nas nossas buscas.

### O que devemos fazer:

- Procurar cada produto no Google Shopping e pegar todos os resultados que tenham preço dentro da faixa e sejam os produtos corretos
- O mesmo para o Buscapé
- Enviar um e-mail para o seu e-mail (no caso da empresa seria para a área de compras por exemplo) com a notificação e a tabela com os itens e preços encontrados, junto com o link de compra. (Vou usar o e-mail pythonimpressionador@gmail.com. Use um e-mail seu para fazer os testes para ver se a mensagem está chegando)

#### Preparar e abrir o Navegador

In [167]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
import email.message
from email import encoders
import pandas as pd
import time

servico = Service(ChromeDriverManager().install())
navegador = webdriver.Chrome(service=servico)

#### Busca no Google

In [168]:
def busca_google(item, evitar, p_min, p_max):
    navegador.get("https://www.google.com/")
    texto_busca = f'"{item}"'
    for palavra in evitar:
        texto_busca += ' -' + palavra
    navegador.find_element(By.CLASS_NAME, 'gLFyf').send_keys(texto_busca)
    navegador.find_element(By.CLASS_NAME, 'gLFyf').send_keys(Keys.ENTER)
    menu = navegador.find_elements(By.CLASS_NAME, 'hdtb-mitem')
    for botao in menu:
        if 'Shopping' in botao.text:
            botao.click()
            break
    dic_item = {}
    lista_dics = []
    try:
        destaque = navegador.find_element(By.CLASS_NAME, 'sh-dp__sc')
        p_mel_cor = destaque.find_element(By.CLASS_NAME, '_-p8').text
        if len(p_mel_cor) == 11:
            p_mel_cor_val = float(p_mel_cor[3:11].replace('.','').replace(',','.'))
            if p_mel_cor_val >= p_min and p_mel_cor_val < p_max:
                dic_item = {}
                dic_item['nome'] = destaque.find_element(By.CLASS_NAME, 'sh-t__title-popout').text
                dic_item['preco'] = p_mel_cor
                dic_item['link'] = destaque.find_element(By.CLASS_NAME, 'sh-t__title-popout').get_attribute('href')
                print(dic_item)
                lista_dics.append(dic_item)
            else:
                print('Preço do destaque fora do intervalo')
        else:
            print('Preço do destaque não tem 11 caracteres')
    except:
        print('Destaque não encontrado')
    anuncios = navegador.find_elements(By.CLASS_NAME, 'i0X6df')
    print(len(anuncios))
    for i, anuncio in enumerate(anuncios):
        preco = anuncio.find_element(By.CLASS_NAME, 'a8Pemb').text
        if len(preco) == 11:
            preco_val = float(preco[3:11].replace('.','').replace(',','.'))
            if preco_val >= p_min and preco_val < p_max:
                dic_item = {}
                dic_item['nome'] = anuncio.find_element(By.CLASS_NAME, 'tAxDx').text
                dic_item['preco'] = preco
                dic_item['link'] = anuncio.find_element(By.CLASS_NAME, 'Lq5OHe').get_attribute('href')
                print(dic_item)
                lista_dics.append(dic_item)
            else:
                print(f'Preço do anuncio {i} fora do intervalo. Preço: {preco}')
        else:
            print(f'Preço do anuncio {i} não tem 11 caracteres. Preço: {preco}')
    return lista_dics

#### Busca no Buscapé

In [169]:
def busca_buscape(item, evitar, p_min, p_max):
    navegador.get("https://www.buscape.com.br/")
    navegador.find_element(By.XPATH, '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(item)
    navegador.find_element(By.XPATH, '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(Keys.ENTER)
    time.sleep(5)
    anuncios = navegador.find_elements(By.CLASS_NAME, 'SearchCard_ProductCard_Inner__7JhKb')
    dic_item = {}
    lista_dics = []
    print(len(anuncios))
    for i, anuncio in enumerate(anuncios):
        nome_prod = anuncio.find_element(By.CLASS_NAME, 'SearchCard_ProductCard_Name__ZaO5o').text
        check_evitar = False
        for palavra in evitar:
            if palavra in nome_prod.lower():
                check_evitar = True
                print(f'Encontramos "{palavra}" no nome do item {nome_prod}.')
        if check_evitar == False:
            preco = anuncio.find_element(By.CLASS_NAME, 'Text_MobileHeadingS__Zxam2').text
            if len(preco) == 11:
                preco_val = float(preco[3:11].replace('.','').replace(',','.'))
                if preco_val >= p_min and preco_val < p_max:
                    dic_item = {}
                    dic_item['nome'] = nome_prod
                    dic_item['preco'] = preco
                    dic_item['link'] = anuncio.get_attribute('href')
                    print(dic_item)
                    lista_dics.append(dic_item)
                else:
                    print(f'Preço do anuncio {i} fora do intervalo. Preço: {preco}')
            else:
                print(f'Preço do anuncio {i} não tem 11 caracteres. Preço: {preco}')
    return lista_dics

#### Enviar email

In [170]:
def enviar_email(produto, tabela_ofertas):
    corpo_email = f'''
    <p>Prezados,</p>
    <p></p>
    <p>Encontramos as seguintes ofertas para o produto {produto} dentro das especificações fornecidas:</p>
    <p>{tabela_ofertas.to_html(index=False)}</p>
    <p></p>
    <p>Estamos à disposição para quaisquer dúvidas.</p>
    <p></p>
    <p>Att.,</p>
    <p>Gabriel de La Rocha</p>
    '''
    
    msg = MIMEMultipart()
    msg['Subject'] = f"Ofertas do Dia - {produto}"
    msg['From'] = 'gabrieldelarocha@gmail.com'
    msg['To'] = 'gabriel.delarocha@consultoriadoamanha.com.br'
    password = 'cbkclecoejvlaopt' 
    msg.add_header('Content-Type', 'html')
    msg.attach(MIMEText(corpo_email, 'html'))
    
    s = smtplib.SMTP('smtp.gmail.com: 587')
    s.starttls()
    # Login Credentials for sending the mail
    s.login(msg['From'], password)
    s.sendmail(msg['From'], [msg['To']], msg.as_string())
    print(f'Email do produto {produto} enviado')

#### Preparar buscas

In [171]:
buscas_df = pd.read_excel('buscas.xlsx')
for i, item in enumerate(buscas_df['Nome']):
    evitar = []
    lista_google = []
    lista_buscape = []
    lista_buscas = []
    texto_evitar = buscas_df.loc[i, 'Termos banidos'].split()
    for palavra in texto_evitar:
        evitar.append(palavra)
    p_min = float(buscas_df.loc[i, 'Preço mínimo'])
    p_max = float(buscas_df.loc[i, 'Preço máximo'])
    lista_google = busca_google(item, evitar, p_min, p_max)
    lista_buscape = busca_buscape(item, evitar, p_min, p_max)
    lista_buscas = lista_google + lista_buscape
    ofertas_df = pd.DataFrame(lista_buscas)
    display(ofertas_df)
    enviar_email(item, ofertas_df)

Destaque não encontrado
61
Preço do anuncio 0 não tem 11 caracteres. Preço: 
Preço do anuncio 1 fora do intervalo. Preço: R$ 1.689,90
Preço do anuncio 2 fora do intervalo. Preço: R$ 4.139,10
Preço do anuncio 3 fora do intervalo. Preço: R$ 4.269,00
{'nome': 'iPhone 12 64GB - Azul - Estou Zerado', 'preco': 'R$ 3.422,51', 'link': 'https://www.google.com/shopping/product/357344828079836893?q=%22iphone+12+64gb%22+-mini+-watch&biw=1036&bih=714&dpr=1.25&prds=eto:2193268644434894717_0,pid:4387230182954649532,rsk:PC_11630641001267601036&sa=X&ved=0ahUKEwiruoKqpoH-AhVzppUCHVbdAwMQ8wIIrAw'}
Preço do anuncio 5 fora do intervalo. Preço: R$ 7.162,00
{'nome': 'iPhone 12 64GB - Roxo - Estou Zerado', 'preco': 'R$ 3.422,51', 'link': 'https://www.google.com/shopping/product/1409198851555036304?q=%22iphone+12+64gb%22+-mini+-watch&biw=1036&bih=714&dpr=1.25&prds=eto:8904915105488796308_0,pid:5070126177312031027&sa=X&ved=0ahUKEwiruoKqpoH-AhVzppUCHVbdAwMQ8wIIzww'}
{'nome': 'Apple Iphone 12 Mini 4gb/64gb 5.4 Az

,nome,preco,link
0,iPhone 12 64GB - Azul - Estou Zerado,"R$ 3.422,51",https://www.google.com/shopping/product/357344...
1,iPhone 12 64GB - Roxo - Estou Zerado,"R$ 3.422,51",https://www.google.com/shopping/product/140919...
2,Apple Iphone 12 Mini 4gb/64gb 5.4 Azul,"R$ 3.469,49",https://www.google.com/url?url=https://www.tra...
3,iPhone 12 Mini 64GB - Preto - Estou Zerado,"R$ 3.217,31",https://www.google.com/shopping/product/121811...
4,Apple iPhone 12 64gb 5g Vitrine Original Bater...,"R$ 3.499,00",https://www.google.com/url?url=https://produto...
5,"(Seminovo) iPhone 12 mini Apple Branco, 64GB","R$ 3.059,15",https://www.google.com/url?url=https://www.taq...
6,"(Seminovo) iPhone 12 Mini Apple Preto, 64GB","R$ 3.059,15",https://www.google.com/url?url=https://www.taq...
7,"IPhone 11 Apple (64GB) Preto Tela 6,1 4G Câmer...","R$ 3.200,00",https://www.google.com/url?url=https://www.sub...
8,Apple Smartphone iPhone 12 4gb/128gb 6.1 Dual ...,"R$ 3.377,99",https://www.google.com/shopping/product/100343...
9,"iPhone 11 Apple 64GB Branco, Tela de 6,1”, Câm...","R$ 3.499,90",https://www.google.com/url?url=https://www.for...


Email do produto iphone 12 64gb enviado
Destaque não encontrado
60
Preço do anuncio 0 fora do intervalo. Preço: R$ 2.399,00
Preço do anuncio 1 fora do intervalo. Preço: R$ 2.159,99
Preço do anuncio 2 fora do intervalo. Preço: R$ 2.339,00
Preço do anuncio 3 fora do intervalo. Preço: R$ 2.799,00
Preço do anuncio 4 fora do intervalo. Preço: R$ 2.499,99
Preço do anuncio 5 fora do intervalo. Preço: R$ 3.652,23
Preço do anuncio 6 fora do intervalo. Preço: R$ 2.413,80
Preço do anuncio 7 fora do intervalo. Preço: R$ 1.701,69
Preço do anuncio 8 fora do intervalo. Preço: R$ 2.334,99
Preço do anuncio 9 fora do intervalo. Preço: R$ 2.479,90
Preço do anuncio 10 fora do intervalo. Preço: R$ 2.659,99
Preço do anuncio 11 fora do intervalo. Preço: R$ 2.676,97
Preço do anuncio 12 fora do intervalo. Preço: R$ 2.778,38
Preço do anuncio 13 fora do intervalo. Preço: R$ 2.236,02
Preço do anuncio 14 não tem 11 caracteres. Preço: R$ 1.690,16 + impostos
Preço do anuncio 15 fora do intervalo. Preço: R$ 3.079,90


,nome,preco,link
0,"PC Gamer U3, Ryzen 5 5600g, GeForce RTX 3060, ...","R$ 4.399,89",https://www.google.com/url?url=https://www.loj...
1,Placa de Video NVIDIA GeForce RTX 3060 12 GB G...,"R$ 4.441,10",https://www.buscape.com.br/placa-de-video/plac...
2,GIGABYTE Placa de vídeo GeForce RTX 3060 Gamin...,"R$ 4.499,00",https://www.buscape.com.br/lead?oid=546338771&...


Email do produto rtx 3060 enviado
